# 数据预处理

## 读取数据集

创建一个人工数据集，存储在CSV中


In [1]:
import os

# 如果不存在时创建data文件夹
os.makedirs(os.path.join('data'), exist_ok=True)
data_file = os.path.join('data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') #列名
    f.write('NA,Pave,127500\n') # 每行一个样本
    f.write("2,NA,106000\n")
    f.write("4,NA,178100\n")
    f.write("NA,NA,140000\n")


pands可以从csv文件中加载原始数据集：**read_csv()**

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

NaN表示缺失值，可以通过**插值法**和**删除法**处理：
- 插值法：用替代值弥补缺失值（一下采用插值法）
- 删除法：忽略缺失值

通过位置索引iloc将data前2列分为inputs，后1列为outputs
inputs的缺失的**数值**用同一列的均值替换

In [4]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于inputs中的类别值或离散值，NaN视作一个类别，则Alley列有两个类别"Pave"和"NaN"
pandas将此列转换成两列"Alley_Pave"和"Alley_NaN"填入01代表存在与否：**get.dummies()**

In [5]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 转换为张量格式

现在inputs和outputs都是数值类型，可以转换成张量格式

In [6]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
Y = torch.tensor(outputs.to_numpy(dtype=float))
(X, Y)

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 练习

删除缺失值最多的列

In [11]:
data = pd.read_csv(data_file)
# .isnull()：转换成逻辑矩阵，缺失为True
# .sum()：按列求和(True为1，False为0，相当于计算缺失值个数)
# .idxmax()：获得最大值的下标
max_id = data.isnull().sum().idxmax()
data_clean = data.drop(columns=[max_id])
data_clean

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000
